In [2]:
# tester env to better understand the dataflow of main.py
import pandas as pd
import numpy as np

from datetime import datetime
from tqdm import tqdm
import os

# from data_prep import Table
# from sketches import Synopsis, Correlation
# from RL_Agent import Autofeature_agent
# from RL_Environment import ISOFAEnvironment

""" Questions:
why drop month and year from flight activity?
"""

' Questions:\nwhy drop month and year from flight activity?\n'

## Basic Attempt at Data Analysis & Cleaning

In [86]:
paths = ["data/Customer Flight Activity.csv", "data/Customer Loyalty History.csv"]
df = pd.read_csv(paths[0])
df

,Loyalty Number,Year,Month,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100590,2018,6,12,15276,22914.0,0,0
1,100590,2018,7,12,9168,13752.0,0,0
2,100590,2018,5,4,6504,9756.0,0,0
3,100590,2018,10,0,0,0.0,512,92
4,100590,2018,2,0,0,0.0,0,0
...,...,...,...,...,...,...,...,...
392931,999986,2018,4,0,0,0.0,0,0
392932,999986,2018,5,0,0,0.0,0,0
392933,999986,2018,6,0,0,0.0,0,0
392934,999986,2018,9,0,0,0.0,0,0


In [87]:
df.describe()

,Loyalty Number,Year,Month,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
count,392936.000000,392936.000000,392936.000000,392936.000000,392936.000000,392936.000000,392936.000000,392936.000000
mean,550527.519034,2017.513661,6.513661,1.294888,1941.440201,2027.172345,31.304263,5.635661
std,258604.580187,0.499814,3.445428,1.962675,3239.975889,3872.139841,126.653775,22.801167
min,100018.000000,2017.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,327688.000000,2017.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,551833.000000,2018.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,772194.000000,2018.000000,10.000000,2.000000,3018.000000,3039.000000,0.000000,0.000000
max,999986.000000,2018.000000,12.000000,28.000000,67284.000000,100926.000000,876.000000,158.000000


In [88]:
# GENERATING TIME SCALE
mf2017 = []  # number of months from 2017
for i, row in tqdm(df.iterrows()):
    diff = int(row["Year"]) - 2017
    num = diff*12 + int(row["Month"])
    mf2017.append(num)
df["mf2017"] = mf2017
df

21164it [00:01, 16384.22it/s]


KeyboardInterrupt: 

In [ ]:
df.sort_values(by=["mf2017"], ascending=False, inplace=True)
ln = df["Loyalty Number"].unique()
df[df["Loyalty Number"]==ln[0]]

,Loyalty Number,Year,Month,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed,mf2017
392935,999986,2018,12,0,0,0.0,0,0,24
392927,999986,2018,11,6,4338,6507.0,0,0,23
392928,999986,2018,10,4,4588,6882.0,0,0,22
392934,999986,2018,9,0,0,0.0,0,0,21
392926,999986,2018,8,8,14680,22020.0,655,118,20
392925,999986,2018,7,20,32340,48510.0,0,0,19
392933,999986,2018,6,0,0,0.0,0,0,18
392932,999986,2018,5,0,0,0.0,0,0,17
392931,999986,2018,4,0,0,0.0,0,0,16
392930,999986,2018,3,1,1069,1603.5,0,0,15


## Trend Visualization

In [ ]:
import plotly.express as px

px.line(df[df["Loyalty Number"].isin(ln[:5])], x="mf2017", y="Distance", color="Loyalty Number")

In [ ]:
px.scatter(df[df["Loyalty Number"]==ln[0]], x="mf2017", y="Distance", trendline="ols")

## Complex Feature Extraction

In [ ]:
extracted_cols = ["Points Accumulated", "Points Redeemed", "Distance", "Total Flights"]
def derive_invariants(df, deg=2):
    """ fancy function that extracts an analysis of each loyalty number via polynomial regression
        and losely returns a dataframe containing its features
        NOTE: this will apply a basic smoothing to the uniqueness of the data which may
            help future analysis
        RUNTIME NOTE: make sure that mf2017 is created first to make the regression smoother """
    new_cols = ["Loyalty Number"]  # making new column names
    for c in extracted_cols:
        for d in range(deg+1):
            new_cols.append(c + " D" + str(d))
    for c in extracted_cols:
        new_cols.append("SUM "+c)
            
    res = []
    for ln in tqdm(df["Loyalty Number"].unique()):
        subdf = df[df["Loyalty Number"]==ln]
        # get polynomial regression based analysis of points & distance traveled
        points_model = np.polyfit(subdf["mf2017"], subdf["Points Accumulated"], deg).tolist()
        redeem_model = np.polyfit(subdf["mf2017"], subdf["Points Redeemed"], deg).tolist()
        dist_model = np.polyfit(subdf["mf2017"], subdf["Distance"], deg).tolist()
        flights_model = np.polyfit(subdf["mf2017"], subdf["Total Flights"], deg).tolist()
        # get absolute statistics
        stats = subdf[extracted_cols].sum().tolist()
        res.append([ln]+points_model+redeem_model+dist_model+flights_model+stats)
    return pd.DataFrame(res, columns=new_cols)

df2 = derive_invariants(df)
df2.to_csv("data/Customer Flight Complex.csv")
df2

100%|██████████| 16737/16737 [00:50<00:00, 331.13it/s]


,Loyalty Number,Points Accumulated D0,Points Accumulated D1,Points Accumulated D2,Points Redeemed D0,Points Redeemed D1,Points Redeemed D2,Distance D0,Distance D1,Distance D2,Total Flights D0,Total Flights D1,Total Flights D2,SUM Points Accumulated,SUM Points Redeemed,SUM Distance,SUM Total Flights
0,999986,-700.832168,27076.031469,-246417.545455,2.639860,-119.087413,1392.272727,-467.221445,18050.687646,-164278.363636,-0.255245,10.017483,-92.000000,88201.5,1199.0,58801.0,40.0
1,981395,-6.840347,300.134752,1059.636364,0.683696,-21.484130,149.005435,-6.840347,300.134752,1059.636364,0.000555,0.043954,1.378953,81954.0,481.0,81954.0,49.0
2,989802,-7.332229,189.695281,2460.472332,-0.298518,11.006858,-45.763340,-7.332229,189.695281,2460.472332,-0.003002,0.131148,1.265316,80032.0,741.0,80032.0,55.0
3,988921,9.043113,-169.557835,2863.003953,-0.988750,25.857890,-69.103755,9.043113,-169.557835,2863.003953,0.002273,-0.066818,2.079545,62156.0,1254.0,62156.0,41.0
4,988307,14.742711,-244.775155,3163.594368,0.000000,0.000000,0.000000,14.742711,-244.775155,3163.594368,0.015704,-0.288249,2.730237,74733.0,0.0,74733.0,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16732,749362,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
16733,750154,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
16734,750153,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
16735,748649,-9.369960,161.576838,2746.073123,-0.208513,8.168483,35.465415,-9.369960,161.576838,2746.073123,-0.006902,0.158632,1.592885,68466.0,2280.0,68466.0,52.0


## Categorical Feature Extraction

In [ ]:
df = pd.read_csv(paths[1])
df

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16732,823768,Canada,British Columbia,Vancouver,V6E 3Z3,Female,College,NaN,Married,Star,61850.19,Standard,2012,12,NaN,NaN
16733,680886,Canada,Saskatchewan,Regina,S1J 3C5,Female,Bachelor,89210.0,Married,Star,67907.27,Standard,2014,9,NaN,NaN
16734,776187,Canada,British Columbia,Vancouver,V5R 1W3,Male,College,NaN,Single,Star,74228.52,Standard,2014,3,NaN,NaN
16735,906428,Canada,Yukon,Whitehorse,Y2K 6R0,Male,Bachelor,-57297.0,Married,Star,10018.66,2018 Promotion,2018,4,NaN,NaN


In [ ]:
# identify categorical columns automnatically and 1hot encode
def derive_categorical(df, ordinal_keys=["month", "year"], verbose=False):
    categorical_cols = []
    for col in df.columns:
        types = list(df[col].unique())
        if len(types) == 1:
            if verbose:
                print("\ncolumn", col, "has only 1 value and therefore is giving no information so we drop it...")
            df.drop(col, axis=1, inplace=True)
        elif len(types) == 2:
            encoding = dict([[name, i] for i, name in enumerate(types)])
            if verbose: print("\nEncoding for column", col, "=>", encoding)
            df[col] = df[col].apply(lambda x: encoding[x])
        elif len(types) < 50:
            is_cat = True
            for word in ordinal_keys:
                if word.lower() in col.lower():
                    is_cat = False
                    break
            if is_cat:
                if verbose:
                    print("\ncolumn", col, "should be unique...")
                    print(types)
                categorical_cols.append(col)
    # one hot multi-category columns
    oh = pd.get_dummies(df[categorical_cols])
    for col in categorical_cols:
        df.drop(col, axis=1, inplace=True)
    return pd.merge(df, oh, left_index=True, right_index=True)
df = derive_categorical(df, ["month", "year"], True)
df.to_csv("data/Customer Loyalty CatEncoded.csv")
df


column Country has only 1 value and therefore is giving no information so we drop it...

column Province should be unique...
['Ontario', 'Alberta', 'British Columbia', 'Quebec', 'Yukon', 'New Brunswick', 'Manitoba', 'Nova Scotia', 'Saskatchewan', 'Newfoundland', 'Prince Edward Island']

column City should be unique...
['Toronto', 'Edmonton', 'Vancouver', 'Hull', 'Whitehorse', 'Trenton', 'Montreal', 'Dawson Creek', 'Quebec City', 'Fredericton', 'Ottawa', 'Tremblant', 'Calgary', 'Thunder Bay', 'Whistler', 'Peace River', 'Winnipeg', 'Sudbury', 'West Vancouver', 'Halifax', 'London', 'Regina', 'Kelowna', "St. John's", 'Victoria', 'Kingston', 'Banff', 'Moncton', 'Charlottetown']

Encoding for column Gender => {'Female': 0, 'Male': 1}

column Education should be unique...
['Bachelor', 'College', 'Master', 'High School or Below', 'Doctor']

column Marital Status should be unique...
['Married', 'Divorced', 'Single']

column Loyalty Card should be unique...
['Star', 'Aurora', 'Nova']

Encoding 

,Loyalty Number,Postal Code,Gender,Salary,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month,...,Education_College,Education_Doctor,Education_High School or Below,Education_Master,Marital Status_Divorced,Marital Status_Married,Marital Status_Single,Loyalty Card_Aurora,Loyalty Card_Nova,Loyalty Card_Star
0,480934,M2Z 4K1,0,83236.0,3839.14,0,2016,2,NaN,NaN,...,0,0,0,0,0,1,0,0,0,1
1,549612,T3G 6Y6,1,NaN,3839.61,0,2016,3,NaN,NaN,...,1,0,0,0,1,0,0,0,0,1
2,429460,V6E 3D9,1,NaN,3839.75,0,2014,7,2018.0,1.0,...,1,0,0,0,0,0,1,0,0,1
3,608370,P1W 1K4,1,NaN,3839.75,0,2013,2,NaN,NaN,...,1,0,0,0,0,0,1,0,0,1
4,530508,J8Y 3Z5,1,103495.0,3842.79,0,2014,10,NaN,NaN,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16732,823768,V6E 3Z3,0,NaN,61850.19,0,2012,12,NaN,NaN,...,1,0,0,0,0,1,0,0,0,1
16733,680886,S1J 3C5,0,89210.0,67907.27,0,2014,9,NaN,NaN,...,0,0,0,0,0,1,0,0,0,1
16734,776187,V5R 1W3,1,NaN,74228.52,0,2014,3,NaN,NaN,...,1,0,0,0,0,0,1,0,0,1
16735,906428,Y2K 6R0,1,-57297.0,10018.66,1,2018,4,NaN,NaN,...,0,0,0,0,0,1,0,0,0,1


## Looking at Additional Data from RL Paper
### School Data

In [ ]:
df = pd.read_csv("data/Schools.csv")
df

,Campus Name,CCSF Entity,Lower Grade,Upper Grade,Grade Range,Category,Map Label,Lower Age,Upper Age,General Type,...,Campus Address,Supervisor District,County FIPS,County Name,Location 1,Neighborhoods (old),Zip Codes,Fire Prevention Districts,Police Districts,Supervisor Districts
0,"Milk, Harvey Milk Childrens Center",SFUSD,-2,-1,PK,USD PreK,CDC095,3,4,CDC,...,"841 ELLIS ST, SAN FRANCISCO CA 94117",6,6075,SAN FRANCISCO,"CA\n(37.783802, -122.420105)",36,28858,7.0,9.0,9
1,Mckinley Elementary School,SFUSD,0,5,K-5,USD Grades K-5,PS075,5,10,PS,...,"1025 14TH ST, San Francisco, CA 94114",8,6075,SAN FRANCISCO,"CA\n(37.766884, -122.436279)",3,28862,15.0,5.0,5
2,Jewish Community Center San Francisco - Rosenb...,Private,-2,-1,PK,Independent / Private,CDC058,3,4,CDC,...,"325 ARGUELLO BLVD, SAN FRANCISCO, CA 94118",1,6075,SAN FRANCISCO,"CA\n(37.784588, -122.459488)",11,54,11.0,6.0,2
3,Eureka Learning Center,Private,-2,-1,PK,Independent / Private,CDC035,3,4,CDC,...,"464 DIAMOND ST, SAN FRANCISCO, CA 94114",8,6075,SAN FRANCISCO,"CA\n(37.754967, -122.437004)",22,28862,2.0,4.0,5
4,Noriega Early Education School,SFUSD,-2,5,PK-5,USD PreK/TK-5,PS085,3,10,PS,...,"1775 44TH AVE, San Francisco, CA 94122",4,6075,SAN FRANCISCO,"CA\n(37.753834, -122.503654)",35,56,1.0,8.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,Golden Gate Guppies Preschool,Private,-2,-1,PK,Independent / Private,CDC046,3,4,CDC,...,"2426 CALIFORNIA ST, SAN FRANCISCO, CA 94115",2,6075,SAN FRANCISCO,"CA\n(37.789124, -122.434509)",27,29490,13.0,9.0,1
441,Lands End School,Private,-2,-1,PK,Independent / Private,CDC071,3,4,CDC,...,"4150 CLEMENT ST BLDG T35, SAN FRANCISCO, CA 94121",1,6075,SAN FRANCISCO,"CA\n(37.782593, -122.504578)",15,55,11.0,6.0,2
442,San Francisco Pacific Academy,Private,0,8,K-8,Independent / Private,IND317,5,13,IND,...,"3301 BALBOA ST, SAN FRANCISCO, CA 94121",1,6075,SAN FRANCISCO,"CA\n(37.775639, -122.494286)",26,55,11.0,6.0,2
443,"Wells, Ida B. Wells High School",SFUSD,9,12,9-12,USD Grades 9-12,PS124,14,17,PS,...,"1099 HAYES ST, SAN FRANCISCO, CA 94117",5,6075,SAN FRANCISCO,"CA\n(37.775013, -122.433899)",10,29492,15.0,5.0,11


In [ ]:
df = derive_categorical(df, ["grade", "age", "zip"], True)
df.to_csv("data/Schools CatEncoded.csv")
df


column CCSF Entity should be unique...
['SFUSD', 'Private', 'SFCCD']

column Category should be unique...
['USD PreK', 'USD Grades K-5', 'Independent / Private', 'USD PreK/TK-5', 'USD Grades 9-12', 'USD Grades 6-8', 'USD PreK/TK', 'Community College District', 'USD PreK, Grades K-5', 'USD PreK-5, Grades K-5', 'USD County School', 'USD PreK-5', 'USD Charter School', 'USD Grades K-8', 'USD Grades PK-8', 'USD PreK/TK-4', 'USD Grades 6-12', 'USD PreK-4, Grades K-5', 'USD PreK-4', 'USD Charter School; PreK-5', 'USD PreK-8', 'USD PreK-5                                   ', 'USD Grades K-5, PreK-5', 'USD Grades K-4']

column General Type should be unique...
['CDC', 'PS', 'IND', 'CC']

column Supervisor District should be unique...
[6, 8, 1, 4, 5, 7, 10, 3, 2, 9, 11]

column County FIPS has only 1 value and therefore is giving no information so we drop it...

column County Name has only 1 value and therefore is giving no information so we drop it...

column Neighborhoods (old) should be uniqu

,Campus Name,Lower Grade,Upper Grade,Grade Range,Map Label,Lower Age,Upper Age,CDS Code,Campus Address,Location 1,...,Category_USD PreK-5,"Category_USD PreK-5, Grades K-5",Category_USD PreK-8,Category_USD PreK/TK,Category_USD PreK/TK-4,Category_USD PreK/TK-5,General Type_CC,General Type_CDC,General Type_IND,General Type_PS
0,"Milk, Harvey Milk Childrens Center",-2,-1,PK,CDC095,3,4,NaN,"841 ELLIS ST, SAN FRANCISCO CA 94117","CA\n(37.783802, -122.420105)",...,0,0,0,0,0,0,0,1,0,0
1,Mckinley Elementary School,0,5,K-5,PS075,5,10,3.868479e+13,"1025 14TH ST, San Francisco, CA 94114","CA\n(37.766884, -122.436279)",...,0,0,0,0,0,0,0,0,0,1
2,Jewish Community Center San Francisco - Rosenb...,-2,-1,PK,CDC058,3,4,3.805047e+08,"325 ARGUELLO BLVD, SAN FRANCISCO, CA 94118","CA\n(37.784588, -122.459488)",...,0,0,0,0,0,0,0,1,0,0
3,Eureka Learning Center,-2,-1,PK,CDC035,3,4,3.868479e+13,"464 DIAMOND ST, SAN FRANCISCO, CA 94114","CA\n(37.754967, -122.437004)",...,0,0,0,0,0,0,0,1,0,0
4,Noriega Early Education School,-2,5,PK-5,PS085,3,10,3.868478e+13,"1775 44TH AVE, San Francisco, CA 94122","CA\n(37.753834, -122.503654)",...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,Golden Gate Guppies Preschool,-2,-1,PK,CDC046,3,4,3.840021e+08,"2426 CALIFORNIA ST, SAN FRANCISCO, CA 94115","CA\n(37.789124, -122.434509)",...,0,0,0,0,0,0,0,1,0,0
441,Lands End School,-2,-1,PK,CDC071,3,4,3.805063e+08,"4150 CLEMENT ST BLDG T35, SAN FRANCISCO, CA 94121","CA\n(37.782593, -122.504578)",...,0,0,0,0,0,0,0,1,0,0
442,San Francisco Pacific Academy,0,8,K-8,IND317,5,13,3.868479e+13,"3301 BALBOA ST, SAN FRANCISCO, CA 94121","CA\n(37.775639, -122.494286)",...,0,0,0,0,0,0,0,0,1,0
443,"Wells, Ida B. Wells High School",9,12,9-12,PS124,14,17,3.868478e+13,"1099 HAYES ST, SAN FRANCISCO, CA 94117","CA\n(37.775013, -122.433899)",...,0,0,0,0,0,0,0,0,0,1


### Air Quality Data

In [89]:
df = pd.read_csv("data/Air_Quality.csv")
df

,Unique ID,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value,Message
0,179772,640,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,409.0,Southeast Queens,2015,01/01/2015,0.3,NaN
1,179785,640,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,209.0,Bensonhurst - Bay Ridge,2015,01/01/2015,1.2,NaN
2,178540,365,Fine particles (PM 2.5),Mean,mcg/m3,UHF42,209.0,Bensonhurst - Bay Ridge,Annual Average 2012,12/01/2011,8.6,NaN
3,178561,365,Fine particles (PM 2.5),Mean,mcg/m3,UHF42,409.0,Southeast Queens,Annual Average 2012,12/01/2011,8.0,NaN
4,823217,365,Fine particles (PM 2.5),Mean,mcg/m3,UHF42,409.0,Southeast Queens,Summer 2022,06/01/2022,6.1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
18020,816914,643,Annual vehicle miles traveled,Million miles,per square mile,CD,503.0,Tottenville and Great Kills (CD3),2019,01/01/2019,12.9,NaN
18021,816913,643,Annual vehicle miles traveled,Million miles,per square mile,CD,503.0,Tottenville and Great Kills (CD3),2010,01/01/2010,14.7,NaN
18022,816872,643,Annual vehicle miles traveled,Million miles,per square mile,UHF42,208.0,Canarsie - Flatlands,2010,01/01/2010,43.4,NaN
18023,816832,643,Annual vehicle miles traveled,Million miles,per square mile,UHF42,407.0,Southwest Queens,2010,01/01/2010,65.8,NaN


In [90]:
df = derive_categorical(df, ["indicator", "date"], True)
df.to_csv("data/Air Quality CatEncoded.csv")
df


column Name should be unique...
['Boiler Emissions- Total SO2 Emissions', 'Fine particles (PM 2.5)', 'Nitrogen dioxide (NO2)', 'Asthma emergency department visits due to PM2.5', 'Annual vehicle miles traveled', 'Respiratory hospitalizations due to PM2.5 (age 20+)', 'Ozone (O3)', 'Asthma hospitalizations due to Ozone', 'Boiler Emissions- Total NOx Emissions', 'Boiler Emissions- Total PM2.5 Emissions', 'Outdoor Air Toxics - Formaldehyde', 'Asthma emergency departments visits due to Ozone', 'Cardiovascular hospitalizations due to PM2.5 (age 40+)', 'Cardiac and respiratory deaths due to Ozone', 'Deaths due to PM2.5', 'Outdoor Air Toxics - Benzene', 'Annual vehicle miles traveled (trucks)', 'Annual vehicle miles traveled (cars)']

column Measure should be unique...
['Number per km2', 'Mean', 'Estimated annual rate (under age 18)', 'Million miles', 'Estimated annual rate (age 18+)', 'Estimated annual rate', 'Annual average concentration', 'Estimated annual rate (age 30+)']

column Measure I

,Unique ID,Indicator ID,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value,Name_Annual vehicle miles traveled,Name_Annual vehicle miles traveled (cars),Name_Annual vehicle miles traveled (trucks),...,"Measure Info_per 100,000 adults","Measure Info_per 100,000 children",Measure Info_per square mile,Measure Info_ppb,Measure Info_Âµg/m3,Geo Type Name_Borough,Geo Type Name_CD,Geo Type Name_Citywide,Geo Type Name_UHF34,Geo Type Name_UHF42
0,179772,640,409.0,Southeast Queens,2015,01/01/2015,0.3,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,179785,640,209.0,Bensonhurst - Bay Ridge,2015,01/01/2015,1.2,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,178540,365,209.0,Bensonhurst - Bay Ridge,Annual Average 2012,12/01/2011,8.6,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,178561,365,409.0,Southeast Queens,Annual Average 2012,12/01/2011,8.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,823217,365,409.0,Southeast Queens,Summer 2022,06/01/2022,6.1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18020,816914,643,503.0,Tottenville and Great Kills (CD3),2019,01/01/2019,12.9,1,0,0,...,0,0,1,0,0,0,1,0,0,0
18021,816913,643,503.0,Tottenville and Great Kills (CD3),2010,01/01/2010,14.7,1,0,0,...,0,0,1,0,0,0,1,0,0,0
18022,816872,643,208.0,Canarsie - Flatlands,2010,01/01/2010,43.4,1,0,0,...,0,0,1,0,0,0,0,0,0,1
18023,816832,643,407.0,Southwest Queens,2010,01/01/2010,65.8,1,0,0,...,0,0,1,0,0,0,0,0,0,1


## Additional Data Preprocessing for Gcloud

In [12]:
gcdata = "data/gc-data/"
gcloud_files = [file for file in os.listdir(gcdata)]
gcdf = [pd.read_csv(gcdata+data) for data in gcloud_files]
for i, df in enumerate(gcdf):
    print(gcloud_files[i])
    print(df)
    print('\n')

temp.csv
              Day  Temperature        f0_        f1_        f2_         f3_  \
0      1990-01-01    36.577706  43.704762   8.933333  23.852381   99.395238   
1      1990-01-02    36.361572  43.485981  12.509346  23.490654   97.883178   
2      1990-01-03    38.360233  45.018605  13.367442  23.572093   98.232558   
3      1990-01-04    38.360426  45.662037  11.254630  23.648148   98.537037   
4      1990-01-05    36.962411  43.568807  11.683486  23.857798   99.408257   
...           ...          ...        ...        ...        ...         ...   
11954  2023-04-26    47.702083  58.850000  14.250000  24.000000  100.000000   
11955  2023-04-27    53.648098  67.300000  13.900000  23.950000   99.800000   
11956  2023-04-28    55.716667  68.350000  12.950000  24.000000  100.000000   
11957  2023-04-29    51.372917  61.200000  12.250000  24.000000  100.000000   
11958  2023-04-30    47.856250  55.400000  15.800000  24.000000  100.000000   

                f4_        f5_            